# Лабораторная работа №1. Метрические алгоритмы классификации

Выполнили: Рутин Василий, Гордеев Станислав

Все лабораторные работы доступны по адресу: https://github.com/Omenstudio/ML_IFMO_LABS

## 1. Постановка задачи

1. На языке Python программно реализовать два метрических алгоритма классификации: Naive Bayes и K Nearest Neighbours

2. Сравнить работу реализованных алгоритмов с библиотечными из scikit-learn

3. Для тренировки, теста и валидации использовать один из предложенных датасетов (либо найти самостоятельно и внести в таблицу)

4. Сформировать краткий отчет (постановка задачи, реализация, эксперимент с данными, полученные характеристики, вывод


## 2. Исходные данные

Задача классификации: определение наличия заболевания (диабет) у женщин.<br>
Датасет доступен по адресу: https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

Количество записей: 768<br>
Количество атрибутов: 8 плюс класс.<br>
Все атрибуты представлены числовыми значениями:
1. Количество беременностей (шт.)
2. Концентрация плазмы глюкозы (ммоль/л)
3. Диастолическое давление (мм. рт. столба)
4. Толщина кожной складки (мм.)
5. Содержание инсулина (мкЕд/мл)
6. Индекс массы тела (кг/м2)
7. Предрасположенность по родословной линии (Diabetes pedigree function)
8. Возраст (лет)
9. Класс: наличие диабета (0, 1)

## 3. Реализация KNN и NaiveBayes

Были реализованы два алгоритма классификации: Наивный Байесовский и Метод K ближайших соседей.

Метод K ближайших соседей.

In [1]:
import csv
import random
import math
import operator
from collections import defaultdict


class MyKNN:

    def __init__(self, n_neighbors=5):
        self.train_inputs = []
        self.train_outputs = []
        self.K = n_neighbors

    def fit(self, inputs, outputs):
        """
        "Обучает" классификатор.
        По факту тупо запоминает значения, чтобы использовать в дальнейшем.
        :param inputs:
        :param outputs:
        :return:
        """
        self.train_inputs = inputs
        self.train_outputs = outputs

    def predict(self, inputs):
        """
        Классифицирует входные данные
        :param inputs:
        :return:
        """
        res = []
        for x in range(len(inputs)):
            neighbors = self.__get_neighbors(inputs[x])
            result = self.select_neighbor(neighbors)
            res.append(result)
        return res

    def __get_neighbors(self, test_object):
        """
        Возвращает список блажайших К соседей
        :param test_object:
        :return:
        """
        distances = []
        length = len(test_object)
        for i in range(len(self.train_inputs)):
            dist = self.dist(test_object, self.train_inputs[i], length)
            distances.append((self.train_outputs[i], dist))
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
        for i in range(self.K):
            neighbors.append(distances[i][0])
        return neighbors

    def dist(self, object_a, object_b, d):
        """
        Находит евклидово расстояние в D-мерном пространстве
        :param object_a:
        :param object_b:
        :param d:
        :return:
        """
        distance = 0
        for i in range(d):
            distance += pow((object_a[i] - object_b[i]), 2)
        return math.sqrt(distance)

    def select_neighbor(self, neighbors):
        """
        Выбирает класс соседей
        :param neighbors:
        :return:
        """
        res_dict = defaultdict(int)
        for x in neighbors:
            res_dict[x] += 1
        res = sorted(res_dict.items(), key=operator.itemgetter(1), reverse=True)
        return res[0][0]

    def score(self, inputs, outputs):
        """
        Классифицирует входные данные и возвращает посчитанную точность по известным выходным данным.
        :param inputs:
        :param outputs:
        :return:
        """
        predicted = self.predict(inputs)
        correct = 0
        for i in range(len(outputs)):
            if outputs[i] == predicted[i]:
                correct += 1
        return correct/float(len(outputs))

    def __str__(self):
        return 'MyKNN with N_neighbors={}'.format(self.K)

Наивный байсовский алгоритм.

In [2]:
import math
from collections import defaultdict


class MyGaussianNaiveBayes:

    def __init__(self):
        self.__summaries = None

    def fit(self, inputs, outputs):
        """
        Обучает классификатор по входным inputs и выходным outputs данным.
        :param inputs:
        :param outputs:
        :return:
        """
        class_dict = self.__combine_by_class(inputs, outputs)
        self.__summaries = {}
        for k, v in class_dict.items():
            self.__summaries[k] = self.__calc_params(v)

    def __combine_by_class(self, inputs, outputs):
        """
        Разделяет обучающую выборку по классам таким образом,
        чтобы можно было получить все элементы, принадлежащие определенному классу.
        :param inputs:
        :param outputs:
        :return: dict[class, elems]
        """
        res = defaultdict(list)
        for i in range(len(inputs)):
            res[outputs[i]].append(inputs[i])
        return res

    def __calc_params(self, dataset):
        """
        :param dataset:
        :return: Среднее значение и среднеквадратичное отклонение для каждого "свойства" входных данных
        """
        return [(self.mean(attribute), self.stdev(attribute)) for attribute in zip(*dataset)]

    def mean(self, mas):
        """
        :param mas:
        :return: среднее значение массива mas
        """
        return sum(mas) / float(len(mas))

    def stdev(self, mas):
        """
        :param numbers:
        :return: среднеквадратичное отклонение в массиве mas
        """
        avg = self.mean(mas)
        variance = sum([pow(x-avg, 2) for x in mas])/float(len(mas)-1)
        return math.sqrt(variance)


    def predict(self, inputs):
        """
        Классифицирует входные данные
        :param inputs:
        :return:
        """
        predictions = []
        for i in range(len(inputs)):
            result = self.__predict_single(inputs[i])
            predictions.append(result)
        return predictions

    def __predict_single(self, input_data):
        """
        Классифицирует один объект.
        :param input_data:
        :return:
        """
        probabilities = self.__calc_allclass_probabilities(input_data)
        className, res = None, -1
        for classValue, probability in probabilities.items():
            if className is None or probability > res:
                res = probability
                className = classValue
        return className

    def __calc_allclass_probabilities(self, input_data):
        """
        Считает вероятности принадлежности объекта ко всем классам
        :param input_data:
        :return:
        """
        res = {}
        for className, classValues in self.__summaries.items():
            res[className] = 1
            for i in range(len(classValues)):
                mean, stdev = classValues[i]
                x = input_data[i]
                res[className] *= self.__calc_single_probability(x, mean, stdev)
        return res


    def __calc_single_probability(self, x, mean, stdev):
        """
        Апостериорная вероятность принадлежности объекта к определенному классу (с предрасчитанными средний и отклонением).
        Магия по формуле.
        :param x:
        :param mean:
        :param stdev:
        :return:
        """
        exponent = math.exp(-(math.pow(x-mean, 2)/(2*math.pow(stdev, 2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

    def score(self, inputs, outputs):
        """
        Классифицирует входные данные, сравнивает с выходными и возвращает точность классификации.
        :param inputs:
        :param outputs:
        :return:
        """
        predicted = self.predict(inputs)
        correct = 0
        for i in range(len(inputs)):
            if outputs[i] == predicted[i]:
                correct += 1
        return correct / float(len(inputs))

    def __str__(self):
        return 'MyGaussianNaiveBayes'

## 4. Сравнение алгоритмов

Для сравнения различных алгоритмов напишем функцию, которая будет загружать выборку и разбивать её на обучающую и тестовую. 

In [3]:
import random

import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


def load_dataset(split_ratio=None, normalize=True):
    dataset_full = np.loadtxt(open("../pima-indians-diabetes.data.csv", "r"), delimiter=",", skiprows=0, dtype=np.float64)
    inputs = dataset_full[:, :-1]
    outputs = dataset_full[:, -1]
    outputs = outputs.astype(np.int64, copy=False)
    if split_ratio is None:
        return inputs, outputs, None, None
    inputs_train, inputs_test, outputs_train, outputs_test \
        = train_test_split(inputs, outputs, test_size=split_ratio, random_state=random.randint(0, 1000))
    if normalize:
        std_scale = preprocessing.StandardScaler().fit(inputs_train)
        inputs_train = std_scale.transform(inputs_train)
        inputs_test = std_scale.transform(inputs_test)
    return inputs_train, outputs_train, inputs_test, outputs_test

Для сравнения алгоритмов выгрузим датасет и подсунем его библиотечным (sci-kit learn) и самописным KNN и NaiveBayes.

In [4]:
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


def load_dataset(split_ratio=None, normalize=True):
    dataset_full = np.loadtxt(open("../pima-indians-diabetes.data.csv", "r"), delimiter=",", skiprows=0, dtype=np.float64)
    inputs = dataset_full[:, :-1]
    outputs = dataset_full[:, -1]
    outputs = outputs.astype(np.int64, copy=False)
    if split_ratio is None:
        return inputs, outputs, None, None
    inputs_train, inputs_test, outputs_train, outputs_test \
        = train_test_split(inputs, outputs, test_size=split_ratio, random_state=random.randint(0, 1000))
    if normalize:
        std_scale = preprocessing.StandardScaler().fit(inputs_train)
        inputs_train = std_scale.transform(inputs_train)
        inputs_test = std_scale.transform(inputs_test)
    return inputs_train, outputs_train, inputs_test, outputs_test

def main():
    # готовим датасет
    inputs_train, outputs_train, inputs_test, outputs_test = load_dataset(split_ratio=.4, normalize=True)
    # готовим классификаторы
    classificators = [
        MyGaussianNaiveBayes(),
        GaussianNB(),
        MyKNN(n_neighbors=75),
        KNeighborsClassifier(n_neighbors=75)
    ]
    # оцениваем работу каждого классификатора
    for clf in classificators:
        clf.fit(inputs_train, outputs_train)
        print(clf, '\nAccuracy: ', clf.score(inputs_test, outputs_test), '\n--------------')

main()

MyGaussianNaiveBayes 
Accuracy:  0.7435064935064936 
--------------
GaussianNB(priors=None) 
Accuracy:  0.714285714286 
--------------
MyKNN with N_neighbors=75 
Accuracy:  0.7045454545454546 
--------------
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=75, p=2,
           weights='uniform') 
Accuracy:  0.704545454545 
--------------


## 5. Заключение

В ходе лабораторной работы были реализованы алгоритмы: наивный байесовский и метод K ближайших соседей.

- Наивный байесовский алгоритм работает лучше на выбранном датасете.
- Реализованный и библиотечный методы К ближайших соседей имеют одинаковую точность, что говорит о том, что алгоритм работает в точности как библиотечный. По крайней мере на представленной выборке.
- Точность реализованного наивного Байеса отличается от библиотечного (~1-4%). Чаще всего в худшую сторону, но бывает, что в лучшую.   